In [19]:
# 📘 1. 파일 불러오기
import json
import os
from collections import defaultdict

base_dir = "./"  # 노트북 기준 상대경로
law_path = os.path.join(base_dir, "../../../../dataset/PIPA/law/law.json")
decree_path = os.path.join(base_dir, "../../../../dataset/PIPA/law/decree.json")

with open(law_path, "r", encoding="utf-8") as f:
    law_data = json.load(f)

with open(decree_path, "r", encoding="utf-8") as f:
    decree_data = json.load(f)


In [20]:
# 🧭 2. 데이터 구조 구축
def build_maps(data):
    id_map = {}
    children = defaultdict(list)
    for item in data:
        id_map[item["id"]] = item
        if item.get("parent"):
            children[item["parent"]].append(item["id"])
    return id_map, children

law_id_map, law_children = build_maps(law_data)
decree_id_map, decree_children = build_maps(decree_data)


In [21]:
# 🔍 3. 하위 항목 재귀 수집
def collect_with_descendants(target_id, id_map, children_map):
    """특정 조항 및 모든 하위 항/호/목을 깊이우선 탐색으로 수집"""
    collected = []

    def dfs(curr_id):
        node = id_map.get(curr_id)
        if node:
            collected.append(node)
            for child in children_map.get(curr_id, []):
                dfs(child)
    dfs(target_id)
    return collected


In [22]:
# 🧠 4. 핵심 함수: find_law_text()

import re

def find_law_text(query: str) -> str:
    """
    query 예시:
      - '개인정보 보호법 제16조'
      - '개인정보 보호법 시행령 제29조의10'
    반환: 문자열 (번호 없이)
    """
    # 데이터 선택
    if "시행령" in query:
        id_map, children_map = decree_id_map, decree_children
        law_name = "개인정보보호법 시행령"
    else:
        id_map, children_map = law_id_map, law_children
        law_name = "개인정보보호법"

    # 조항 ID 추출
    match = re.search(r"(제[\d가-힣]+조(?:의\d+)?)", query)
    if not match:
        return "⚠️ 조항 번호를 찾을 수 없습니다."

    target_id = match.group(1)

    # 존재 확인
    if target_id not in id_map:
        return f"⚠️ '{target_id}' 조항을 찾을 수 없습니다."

    # 결과 수집
    results = collect_with_descendants(target_id, id_map, children_map)
    if not results:
        return f"⚠️ '{target_id}' 조항의 내용을 찾을 수 없습니다."

    # 문자열 조립
    output_lines = []
    for item in results:
        # 조 단위면 법명과 함께 표시
        if item.get("class") == "조":
            output_lines.append(f"<{law_name} {item['id']} ({item['title']})>")
        else:
            output_lines.append(f"{item['id']} : {item['content']}")

    return "\n".join(output_lines)


In [23]:
# 🧪 5. 테스트

print(find_law_text("개인정보 보호법 제3조"))
print("\n" + "="*80 + "\n")
print(find_law_text("개인정보 보호법 시행령 제16조"))
print("\n" + "="*80 + "\n")
print(find_law_text("개인정보 보호법 시행령 제29조의10"))


<개인정보보호법 제3조 (개인정보 보호 원칙)>
제3조 제1항 : 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 범위 에서 최소한의 개인정보만을 적법하고 정당하게 수집하여야 한다.
제3조 제2항 : 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 적합하게 개인정보를 처리하여야 하며, 그 목적 외 의 용도로 활용하여서는 아니 된다.
제3조 제3항 : 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전성 및 최신성이 보장되도록 하여야 한다.
제3조 제4항 : 개인정보처리자는 개인정보의 처리 방법 및 종류 등에 따라 정보주체의 권리가 침해받을 가능성과 그 위험 정도 를 고려하여 개인정보를 안전하게 관리하여야 한다.
제3조 제5항 : 개인정보처리자는 제30조에 따른 개인정보 처리방침 등 개인정보의 처리에 관한 사항을 공개하여야 하며, 열람 청구권 등 정보주체의 권리를 보장하여야 한다.
제3조 제6항 : 개인정보처리자는 정보주체의 사생활 침해를 최소화하는 방법으로 개인정보를 처리하여야 한다.
제3조 제7항 : 개인정보처리자는 개인정보를 익명 또는 가명으로 처리하여도 개인정보 수집목적을 달성할 수 있는 경우 익명 처리가 가능한 경우에는 익명에 의하여, 익명처리로 목적을 달성할 수 없는 경우에는 가명에 의하여 처리될 수 있도 록 하여야 한다.
제3조 제8항 : 개인정보처리자는 이 법 및 관계 법령에서 규정하고 있는 책임과 의무를 준수하고 실천함으로써 정보주체의 신 뢰를 얻기 위하여 노력하여야 한다.


<개인정보보호법 시행령 제16조 (개인정보의 파기방법)>
제16조 제1항 : 개인정보처리자는 법 제21조에 따라 개인정보를 파기할 때에 는 다음 각 호의 구분에 따른 방법으로 해야 한다.
제16조 제1항 제1호 : 전자적 파일 형태인 경우: 복원이 불가능한 방법으로 영구 삭제. 다만, 기술적 특성으로 영구 삭제가 현저히 곤란한 경우에는 법 제58조의2에 해당하는 정보로 처리하여 복원이 불 가능하도록 조치해야

In [24]:
# 📦 6. GPT-5를 이용해 위반 조항별 compliance 판단 수행
from openai import OpenAI
from tqdm import tqdm
import json
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

# few-shot 예시 (이전 메시지에서 만든 3개)
fewshot_examples = [
    {
        "role": "user",
        "content": json.dumps({
            "content": "B사는 고객의 이름과 연락처를 이벤트 응모 시 수집했으나, 동의서에 명확한 수집 목적과 보유 기간을 명시하지 않았다. 또한 수집된 정보 일부를 제3자 마케팅 회사에 제공했다.",
            "articles": [
                "개인정보 보호법 제15조",
                "개인정보 보호법 제17조",
                "개인정보 보호법 제30조"
            ]
        }, ensure_ascii=False)
    },
    {
        "role": "assistant",
        "content": json.dumps([
            {
                "id": "개인정보 보호법 제15조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제15조 제1항 제1호", "reason": "정보주체의 명확한 동의를 받지 않고 개인정보를 수집하여 수집 요건을 위반함."},
                    {"id": "제15조 제2항 제1호", "reason": "수집 목적과 보유 기간을 고지하지 않아 동의의 유효성이 인정되지 않음."}
                ]
            },
            {
                "id": "개인정보 보호법 제17조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제17조 제1항 제1호", "reason": "정보주체의 동의 없이 제3자에게 개인정보를 제공하여 제공 요건을 위반함."}
                ]
            },
            {
                "id": "개인정보 보호법 제30조",
                "compliance": True,
                "specific_paragraphs": []
            }
        ], ensure_ascii=False)
    },
    {
        "role": "user",
        "content": json.dumps({
            "content": "C병원은 환자의 진료기록을 클라우드 서버에 보관하면서 암호화 절차를 생략하였다. 하지만 정보주체에게는 접근 제한 조치를 적절히 유지했다.",
            "articles": [
                "개인정보 보호법 제29조",
                "개인정보 보호법 시행령 제30조",
                "개인정보 보호법 제4조"
            ]
        }, ensure_ascii=False)
    },
    {
        "role": "assistant",
        "content": json.dumps([
            {
                "id": "개인정보 보호법 제29조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제29조 본문", "reason": "암호화 등 안전성 확보 조치를 취하지 않아 개인정보의 기술적 보호조치를 위반함."}
                ]
            },
            {
                "id": "개인정보 보호법 시행령 제30조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제30조 제1항", "reason": "클라우드 저장 시 암호화 및 접근 통제 절차를 생략하여 시행령상 보호조치 기준을 준수하지 않음."}
                ]
            },
            {
                "id": "개인정보 보호법 제4조",
                "compliance": True,
                "specific_paragraphs": []
            }
        ], ensure_ascii=False)
    },
    {
        "role": "user",
        "content": json.dumps({
            "content": "D온라인몰은 회원 탈퇴 요청을 받은 후에도 3개월 이상 구매내역을 삭제하지 않았으며, 개인정보 파기 절차가 내부 규정에 따라 즉시 이루어지지 않았다. 다만 개인정보 유출이나 제3자 제공은 없었다.",
            "articles": [
                "개인정보 보호법 제21조",
                "개인정보 보호법 시행령 제16조",
                "개인정보 보호법 제17조"
            ]
        }, ensure_ascii=False)
    },
    {
        "role": "assistant",
        "content": json.dumps([
            {
                "id": "개인정보 보호법 제21조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제21조 제1항", "reason": "정보주체의 요청에도 개인정보를 지체 없이 파기하지 않아 보유기간 제한 원칙을 위반함."}
                ]
            },
            {
                "id": "개인정보 보호법 시행령 제16조",
                "compliance": False,
                "specific_paragraphs": [
                    {"id": "제16조 제2항", "reason": "파기 절차를 내부 규정에 명시하지 않거나 즉시 이행하지 않아 시행령상 파기 기준 위반."}
                ]
            },
            {
                "id": "개인정보 보호법 제17조",
                "compliance": True,
                "specific_paragraphs": []
            }
        ], ensure_ascii=False)
    }
]

In [28]:
# 📑 7. illegal_top16.jsonl 로드 및 GPT-5 호출 (오류 발생 시 즉시 중단)
from tqdm import tqdm
import json
import os
from openai import OpenAI

client = OpenAI()

input_path = os.path.join(base_dir, "illegal_top16.jsonl")
output_path = os.path.join(base_dir, "illegal_top16_result.jsonl")

results = []

with open(input_path, "r", encoding="utf-8") as f:
    lines = [json.loads(line) for line in f]

for data in tqdm(lines, desc="Analyzing with GPT-5 (reasoning: medium)"):
    content = data.get("content", "")
    violated_articles = data.get("violated_articles", [])
    if isinstance(violated_articles, str):
        violated_articles = [violated_articles]

    # ✅ GPT 입력 메시지 구성
    messages = [
        *fewshot_examples,
        {
            "role": "user",
            "content": json.dumps({
                "content": content,
                "articles": violated_articles
            }, ensure_ascii=False)
        },
        {
            "role": "system",
            "content": (
                "위 조항들을 기반으로 각 조항의 준수 여부를 판단하라. "
                "각 조항에 대해 id, compliance, specific_paragraphs를 포함한 JSON 리스트만 출력하라. "
                "compliance가 true인 경우 specific_paragraphs는 빈 리스트로 두어라."
            )
        }
    ]

    # 🧠 GPT-5 호출 (reasoning effort = medium)
    response = client.responses.create(
        model="gpt-5",
        reasoning={ "effort": "medium" },
        text={ "verbosity": "low" },
        input=messages,
    )

    # GPT 출력 처리
    gpt_output = response.output_text.strip()
    parsed_json = json.loads(gpt_output)
    data["gpt_result"] = parsed_json
    results.append(data)


Analyzing with GPT-5 (reasoning: medium): 100%|██████████| 409/409 [1:32:52<00:00, 13.62s/it]


In [ ]:
# 💾 8. illegal_top16_result.jsonl 저장
with open(output_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ 완료: {len(results)}개의 사례가 분석되어 '{output_path}'에 저장되었습니다.")


✅ 완료: 409개의 사례가 분석되어 './illegal_top16_result.jsonl'에 저장되었습니다.
